In [9]:
# Cell 1 - Install required libraries (run once in Colab)
!pip install cryptography pycryptodome -q


In [10]:
# Cell 2 - Import libraries and helper functions
import os
import json
import base64
import time
from datetime import datetime
import random

# Using AES-GCM from the cryptography library (provides confidentiality + integrity)
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

# Helper function to generate random sensor readings
def generate_sensor_reading():
    # Temperature between 15 and 40 degrees Celsius (random)
    temp = round(random.uniform(15.0, 40.0), 2)
    # Humidity between 10% and 90%
    hum = round(random.uniform(10.0, 90.0), 2)
    timestamp = datetime.utcnow().isoformat() + "Z"
    return {"temperature": temp, "humidity": hum, "timestamp": timestamp}


In [11]:
# Cell 3 - Encryption and decryption functions using AES-GCM
# Note: AES-GCM provides confidentiality and authenticated encryption (AEAD)

def generate_key():
    # Generate a 256-bit AES key
    return AESGCM.generate_key(bit_length=256)

def encrypt_payload(aes_key: bytes, payload: dict):
    aesgcm = AESGCM(aes_key)
    # Random nonce (12 bytes recommended for GCM mode)
    nonce = os.urandom(12)
    # Convert payload to JSON and then to bytes
    plaintext = json.dumps(payload, ensure_ascii=False).encode('utf-8')
    # Encrypt (AAD left as None; can be added if needed)
    ciphertext = aesgcm.encrypt(nonce, plaintext, None)
    # Return encrypted data encoded in base64 for easy transmission
    return {
        "nonce_b64": base64.b64encode(nonce).decode(),
        "ciphertext_b64": base64.b64encode(ciphertext).decode()
    }

def decrypt_payload(aes_key: bytes, enc_obj: dict):
    aesgcm = AESGCM(aes_key)
    nonce = base64.b64decode(enc_obj["nonce_b64"])
    ciphertext = base64.b64decode(enc_obj["ciphertext_b64"])
    plaintext = aesgcm.decrypt(nonce, ciphertext, None)
    return json.loads(plaintext.decode('utf-8'))


In [12]:
# Cell 4 - Simulating the IoT device and the server (simple transmission demo)
# Here we generate a reading, encrypt it, then simulate sending it to the server for decryption

# Generate a shared key (in real IoT systems, this is securely injected during the Secure Key Injection phase)
shared_key = generate_key()

# Simulate several transmitted messages
messages = []
for i in range(4):
    payload = generate_sensor_reading()
    enc = encrypt_payload(shared_key, payload)
    # In real systems: this encrypted payload would be transmitted over MQTT/HTTPS/CoAP
    messages.append({"device_payload_plain": payload, "encrypted": enc})
    time.sleep(0.5)  # Small delay to simulate periodic sensor transmission

# Display data before and after encryption (for demonstration)
for idx, msg in enumerate(messages, 1):
    print(f"--- Message {idx} ---")
    print("Data before encryption:")
    print(msg["device_payload_plain"])
    print("\nEncrypted packet (Base64 encoded):")
    print(msg["encrypted"])
    print("\nServer decrypts and displays:")
    decrypted = decrypt_payload(shared_key, msg["encrypted"])
    print(decrypted)
    print("\n")


/tmp/ipython-input-1395911256.py:18: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().isoformat() + "Z"


--- Message 1 ---
Data before encryption:
{'temperature': 32.05, 'humidity': 53.31, 'timestamp': '2025-11-30T05:53:36.510316Z'}

Encrypted packet (Base64 encoded):
{'nonce_b64': 'Qlcgz5XE6jC0UMML', 'ciphertext_b64': '0NMfhDuFYUnYOgbyoVTT2jF4uQd6uQNO4+s7wXbf9cpBr4dwLqgb7qkKkwVfDC9IapFzhg8jEQORsPcayFQ6Fw+Z5hU2OUdO86N716MAO5PMEOwx67TFaRwY26Oy1aKoSXAMhtw='}

Server decrypts and displays:
{'temperature': 32.05, 'humidity': 53.31, 'timestamp': '2025-11-30T05:53:36.510316Z'}


--- Message 2 ---
Data before encryption:
{'temperature': 28.54, 'humidity': 79.05, 'timestamp': '2025-11-30T05:53:37.010679Z'}

Encrypted packet (Base64 encoded):
{'nonce_b64': 'K7G1mw704XtzMLJZ', 'ciphertext_b64': 'OkZJHEr98RGAJG+VQDfosesnwABlIQotRdLKA9MXslFV81Te76mI0b7FA9kvtawDBIyNphEZxZmoXIOQU/adnzl9bwvn6MQTnTcaCpji5iCjZ1NpZh8Znl4823/6aUHu+BGI0V8='}

Server decrypts and displays:
{'temperature': 28.54, 'humidity': 79.05, 'timestamp': '2025-11-30T05:53:37.010679Z'}


--- Message 3 ---
Data before encryption:
{'temper

In [15]:
# Cell Part II - محاكاة دورة الحياة الأمنية لجهاز IoT
import logging
from datetime import datetime
import hashlib
import random
import time

# تهيئة الـlogger ليطبع الطوابع الزمنية
logger = logging.getLogger("iot_lifecycle")
logger.setLevel(logging.INFO)
if not logger.handlers:
    ch = logging.StreamHandler()
    formatter = logging.Formatter('[%(asctime)s] %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    ch.setFormatter(formatter)
    logger.addHandler(ch)

def log_stage(stage_no, message):
    logger.info(f"[Stage {stage_no}] {message}")

# Stage 1: Threat Modeling (نموذج التهديد)
def threat_modeling(device_profile):
    # مثال بسيط: نحدد تهديدات محتملة تبعًا لنوع الجهاز والاتصال
    threats = []
    if device_profile.get("connectivity") == "internet":
        threats.append("Remote exploitation")
    if device_profile.get("has_firmware_update", True):
        threats.append("Malicious firmware via OTA")
    threats.append("Physical tampering")
    time.sleep(0.5)
    return threats

# Stage 2: Secure Boot Initialization (محاكاة)
def secure_boot_check(stored_hash, firmware_blob):
    # نحسب هاش للـfirmware ونقارنه مع المخزن
    h = hashlib.sha256(firmware_blob.encode()).hexdigest()
    time.sleep(0.3)
    return h == stored_hash

# Stage 3: Secure Key Injection (Mock)
def inject_keys():
    # سننشئ "مفاتيح" وهمية (في الواقع يجب أن يتم بحقنها في عنصر آمن)
    keys = {
        "device_id": f"dev-{random.randint(1000,9999)}",
        "aes_key": base64.b64encode(os.urandom(32)).decode()
    }
    time.sleep(0.2)
    return keys

# Stage 4: OTA Firmware Update Check (محاكاة)
def ota_check(current_version, available_version):
    time.sleep(0.3)
    return available_version != current_version, available_version

# Stage 5: Secure Decommissioning
def decommission(keys):
    # نمسح المفاتيح (محاكاة)
    keys.clear()
    time.sleep(0.2)
    return True

# تنفيذ تسلسل المحاكاة مع سجل زمني
device_profile = {"connectivity": "internet", "has_firmware_update": True}
log_stage(1, "Starting threat modeling...")
threats = threat_modeling(device_profile)
log_stage(1, f"Threat model created: {threats}")

log_stage(2, "Verifying secure boot...")
# نفترض أن المخزن يحتوي هاش للـfirmware
firmware_blob = "firmware_v1_binary_content_example"
stored_hash = hashlib.sha256(firmware_blob.encode()).hexdigest()
boot_ok = secure_boot_check(stored_hash, firmware_blob)
if boot_ok:
    log_stage(2, "Secure boot verified.")
else:
    log_stage(2, "Secure boot FAILED! Abort boot.")

log_stage(3, "Injecting keys securely (mock)...")
injected_keys = inject_keys()
log_stage(3, f"Keys injected securely: device_id={injected_keys.get('device_id')}")

log_stage(4, "Checking OTA firmware update...")
current_version = "v1.0"
available_version = "v1.1"  # محاكاة وجود تحديث
needs_update, new_ver = ota_check(current_version, available_version)
if needs_update:
    log_stage(4, f"OTA update available: {new_ver} -> starting verification and update process (simulated).")
    # هنا يمكننا محاكاة تنزيل وتحقق الهاش ثم الاستبدال
    current_version = new_ver
    log_stage(4, f"OTA update applied. Current version: {current_version}")
else:
    log_stage(4, "No OTA update needed.")

log_stage(5, "Decommissioning device, wiping secrets...")
decom_ok = decommission(injected_keys)
if decom_ok:
    log_stage(5, "Device decommissioned, secrets wiped.")
else:
    log_stage(5, "Decommissioning failed.")


[2025-11-30 06:13:16] [Stage 1] Starting threat modeling...
INFO:iot_lifecycle:[Stage 1] Starting threat modeling...
[2025-11-30 06:13:17] [Stage 1] Threat model created: ['Remote exploitation', 'Malicious firmware via OTA', 'Physical tampering']
INFO:iot_lifecycle:[Stage 1] Threat model created: ['Remote exploitation', 'Malicious firmware via OTA', 'Physical tampering']
[2025-11-30 06:13:17] [Stage 2] Verifying secure boot...
INFO:iot_lifecycle:[Stage 2] Verifying secure boot...
[2025-11-30 06:13:17] [Stage 2] Secure boot verified.
INFO:iot_lifecycle:[Stage 2] Secure boot verified.
[2025-11-30 06:13:17] [Stage 3] Injecting keys securely (mock)...
INFO:iot_lifecycle:[Stage 3] Injecting keys securely (mock)...
[2025-11-30 06:13:17] [Stage 3] Keys injected securely: device_id=dev-3139
INFO:iot_lifecycle:[Stage 3] Keys injected securely: device_id=dev-3139
[2025-11-30 06:13:17] [Stage 4] Checking OTA firmware update...
INFO:iot_lifecycle:[Stage 4] Checking OTA firmware update...
[2025-11-